# Importar bibliotecas

In [1]:
import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 200

# Ler o estoque

In [5]:
patt = './Estoque/*/*Estoque*.csv'
list_files = glob(patt)
list_files

['./Estoque\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte1.csv',
 './Estoque\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte2.csv',
 './Estoque\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte3.csv',
 './Estoque\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25\\FIDC FCT CONSIGNADO II  - Estoque 14.07.25-Parte4.csv']

In [6]:
file = list_files[0]

In [7]:
# Colunas que contêm datas.
colunas_data = [
    'DataEmissao',
    'DataAquisicao',
    'DataVencimento',
    'DataGeracao'
]

colunas_texto = [
    'Situacao', 'PES_TIPO_PESSOA', 'CedenteCnpjCpf', 'TIT_CEDENTE_ENT_CODIGO',
    'CedenteNome', 'Cnae', 'SecaoCNAEDescricao', 'NotaPdd', 'SAC_TIPO_PESSOA',
    'SacadoCnpjCpf', 'SacadoNome', 'IdTituloVortx', 'TipoAtivo', 'NumeroBoleto',
    'NumeroTitulo', 'CampoChave', 'PagamentoParcial', 'Coobricacao',
    'CampoAdicional1', 'CampoAdicional2', 'CampoAdicional3', 'CampoAdicional4',
    'CampoAdicional5', 'IdTituloVortxOriginador', 'Registradora',
    'IdContratoRegistradora', 'IdTituloRegistradora', 'CCB', 'Convênio'
]

#*Numéricas: 
colunas_numericas = [
    'ValorAquisicao', 'ValorNominal', 'ValorPresente', 'PDDNota', 'PDDVencido',
    'PDDTotal', 'PDDEfeitoVagao', 'PercentagemEfeitoVagao'
]

# dic pra ler as de texto
dtype_texto = {col: str for col in colunas_texto}


In [8]:
%%time
dfs = []
for file in list_files:
    df_ = pd.read_csv(file, sep=';', encoding='latin1', dtype=dtype_texto, 
                      decimal=',', parse_dates=colunas_data, dayfirst=True)
    dfs.append(df_)

CPU times: total: 32.2 s
Wall time: 1min 12s


In [17]:
# for file in list_files:
#     with open(file, 'r', encoding='latin1') as arquivo:
#         num_linhas = sum(1 for linha in arquivo)
#     print(file, num_linhas)

In [9]:
%%time
df_final = pd.concat(dfs)

CPU times: total: 5.58 s
Wall time: 14.8 s


In [10]:
# criar colunas auxiliares
df_final['_ValorLiquido'] = df_final['ValorPresente'] - df_final['PDDTotal']
df_final['_ValorVencido'] = (df_final['DataVencimento'] <= df_final['DataGeracao']).astype('int') * df_final['ValorPresente']

# sacados com muitos contratos
sacado_contratos = df_final.groupby('SacadoNome')['CCB'].nunique()
k = 3 # numero alto
mask = sacado_contratos[sacado_contratos >= k]
sacado_contratos_alto = sacado_contratos.index[mask]
df_final['_MuitosContratos'] = df_final['SacadoNome'].isin(sacado_contratos_alto).astype('str')

# sacados com muitos entes
sacados_entes = df_final.groupby('SacadoCnpjCpf')['Convênio'].nunique()
k2 = 3
mask = sacados_entes >= k2
sacados_entes_alto = sacados_entes.index[mask]
df_final['_MuitosEntes'] = df_final['SacadoCnpjCpf'].isin(sacados_entes_alto).astype('str')


In [11]:
df_final.memory_usage(deep=True).sum()/1024**2

3027.2806911468506

In [12]:
# Tamanho do estoque
print(f'{df_final["ValorPresente"].sum():_.2f}')

199_232_147.56


# Analisar com `value_counts()`
### Comentários / dúvidas
1. [Situacao] - diferença entre sem cobrança e aditado
1. [SAC_TIPO_PESSOA] - tipo J = jurídico?? tem isso?
1. [SacadoCnpjCpf] - por que tem CNPJ?
1. [SacadoNome] 'BMP SOCIEDADE DE CREDITO DIRETO S.A'
1. [TipoAtivo] - CCB e Contrato. Por que tem contrato?
1. [DataGeracao] - data de referência ou data de processamento?
1. [SacadoCnpjCpf] - tem sacado com 1040 linhas (!)
1. [Convênio] - sacados com muitos convênios (3)
1. Sacado tipo J está com PDD altíssima
1. Por que tem parcelas com valor de aquisicação maior do que valor presente?

In [13]:
df_final2 = df_final[~df_final['Situacao'].isna()].copy()

for col in df_final2.columns:
    if not df_final2[col].dtype == df_final2['Situacao'].dtype:
        continue
    print(df_final2[col].value_counts(dropna=False))
    print('*'*80)
    

Situacao
Sem cobranÃ§a    1471376
Aditado            50939
Name: count, dtype: int64
********************************************************************************
PES_TIPO_PESSOA
J    1522315
Name: count, dtype: int64
********************************************************************************
CedenteCnpjCpf
34.337.707/0001-00    1519907
92.874.270/0001-40       2408
Name: count, dtype: int64
********************************************************************************
TIT_CEDENTE_ENT_CODIGO
318853     1519907
3224693       2408
Name: count, dtype: int64
********************************************************************************
CedenteNome
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A    1519907
BANCO DIGIMAIS S.A.                                  2408
Name: count, dtype: int64
********************************************************************************
Cnae
6499999    1519907
6422100       2408
Name: count, dtype: int64
********************************************

# Verificando as 117.984 linhas com dados nulos
Pela análise abaixo, não tem influência no cálculo

In [14]:
df_final[df_final['Situacao'].isna()].apply(lambda x: (min(x), max(x)))

,Situacao,PES_TIPO_PESSOA,CedenteCnpjCpf,TIT_CEDENTE_ENT_CODIGO,CedenteNome,Cnae,SecaoCNAEDescricao,NotaPdd,SAC_TIPO_PESSOA,SacadoCnpjCpf,SacadoNome,IdTituloVortx,TipoAtivo,DataEmissao,DataAquisicao,DataVencimento,NumeroBoleto,NumeroTitulo,CampoChave,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PagamentoParcial,Coobricacao,DataGeracao,PDDTotal,CampoAdicional1,CampoAdicional2,CampoAdicional3,CampoAdicional4,CampoAdicional5,PDDEfeitoVagao,PercentagemEfeitoVagao,IdTituloVortxOriginador,Registradora,IdContratoRegistradora,IdTituloRegistradora,CCB,Convênio,_ValorLiquido,_ValorVencido,_MuitosContratos,_MuitosEntes
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [40]:
v1_ = df_final['ValorPresente'].sum() 
v2_ = df_final[~df_final['Situacao'].isna()]['ValorPresente'].sum()
np.isclose(v1_, v2_)

True

In [45]:
del df_final

# Sacado == 'J'
aqui não tem problema

In [18]:
df_sacado_J = df_final2[df_final2['SAC_TIPO_PESSOA'] == 'J'].sample(5)

In [19]:
df_sacado_J['SacadoCnpjCpf'].map(len).unique()

array([14], dtype=int64)

# Sacado com CNPJ

In [20]:
set(df_final2['SacadoCnpjCpf'].apply(len))

{14, 18}

In [21]:
df_final2_cnpj = df_final2[df_final2['SacadoCnpjCpf'].map(len)==18]
df_final2_cnpj['SacadoNome'].unique()

array(['BMP SOCIEDADE DE CREDITO DIRETO S.A'], dtype=object)

In [38]:
# for col in df_final2_cnpj.columns:
#     if not df_final2_cnpj[col].dtype == df_final2_cnpj['Situacao'].dtype:
#         continue
#     print(df_final2_cnpj[col].value_counts(dropna=False))
#     print('*'*80)

In [23]:
df_final2_cnpj.describe(include=[np.number])

,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PDDTotal,PDDEfeitoVagao,PercentagemEfeitoVagao,_ValorLiquido,_ValorVencido
count,95.000000,9.500000e+01,95.000000,95.0,95.000000,95.000000,95.000000,95.0,95.0,95.000000
mean,23.177368,5.613000e+01,27.604434,0.0,1.650542,27.604434,27.604434,1.0,0.0,4.135895
std,13.054230,1.428624e-14,15.121244,0.0,8.263526,15.121244,15.121244,0.0,0.0,14.742111
min,7.310000,5.613000e+01,8.795000,0.0,0.000000,8.795000,8.795000,1.0,0.0,0.000000
25%,12.005000,5.613000e+01,14.445900,0.0,0.000000,14.445900,14.445900,1.0,0.0,0.000000
50%,19.710000,5.613000e+01,23.720600,0.0,0.000000,23.720600,23.720600,1.0,0.0,0.000000
75%,32.350000,5.613000e+01,38.939150,0.0,0.000000,38.939150,38.939150,1.0,0.0,0.000000
max,53.070000,5.613000e+01,56.130000,0.0,56.130000,56.130000,56.130000,1.0,0.0,56.130000


In [24]:
df_final2_cnpj.select_dtypes(include=[np.number]).sum()

ValorAquisicao            2201.8500
ValorNominal              5332.3500
ValorPresente             2622.4212
PDDNota                      0.0000
PDDVencido                 156.8015
PDDTotal                  2622.4212
PDDEfeitoVagao            2622.4212
PercentagemEfeitoVagao      95.0000
_ValorLiquido                0.0000
_ValorVencido              392.9100
dtype: float64

In [83]:
# Exportar para excel
df_final2_cnpj.to_excel('df_final2_cnpj.xlsx')

# SacadoCnpjCpf - verificar consistência dos CPFs

In [25]:
def validar_cpf(cpf):
    # Remove caracteres não numéricos
    cpf = ''.join(filter(str.isdigit, str(cpf)))
    
    # Verifica se tem 11 dígitos
    if len(cpf) != 11:
        return False
    
    # Verifica se todos os dígitos são iguais (ex: 111.111.111-11)
    if cpf == cpf[0] * 11:
        return False
    
    # Cálculo do primeiro dígito verificador
    soma = 0
    for i in range(9):
        soma += int(cpf[i]) * (10 - i)
    
    resto = (soma * 10) % 11
    if resto == 10 or resto == 11:
        resto = 0
    if resto != int(cpf[9]):
        return False
    
    # Cálculo do segundo dígito verificador
    soma = 0
    for i in range(10):
        soma += int(cpf[i]) * (11 - i)
    
    resto = (soma * 10) % 11
    if resto == 10 or resto == 11:
        resto = 0
    if resto != int(cpf[10]):
        return False
    
    return True

In [31]:
df_final2['CPF_válido'] = df_final2['SacadoCnpjCpf'].apply(validar_cpf)

In [37]:
df_final2[~df_final2['CPF_válido']]['SacadoCnpjCpf'].unique()

array(['34.337.707/0001-00'], dtype=object)

In [51]:
df_final2.sample(2)

,Situacao,PES_TIPO_PESSOA,CedenteCnpjCpf,TIT_CEDENTE_ENT_CODIGO,CedenteNome,Cnae,SecaoCNAEDescricao,NotaPdd,SAC_TIPO_PESSOA,SacadoCnpjCpf,SacadoNome,IdTituloVortx,TipoAtivo,DataEmissao,DataAquisicao,DataVencimento,NumeroBoleto,NumeroTitulo,CampoChave,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PagamentoParcial,Coobricacao,DataGeracao,PDDTotal,CampoAdicional1,CampoAdicional2,CampoAdicional3,CampoAdicional4,CampoAdicional5,PDDEfeitoVagao,PercentagemEfeitoVagao,IdTituloVortxOriginador,Registradora,IdContratoRegistradora,IdTituloRegistradora,CCB,Convênio,_ValorLiquido,_ValorVencido,_MuitosContratos,_MuitosEntes,CPF_válido
526302,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,J,279.483.072-49,MARIO NAZARENO GOMES BASTOS,44747728,CCB,2024-06-27,2024-06-28,2027-02-20,NaN,3728551931,172599724,34.78,72.0,46.3157,0.0,0.0,NAO,NAO,2025-07-14,31.9799,NaN,NaN,NaN,NaN,NaN,31.979888,0.0,69121725,NaN,NaN,NaN,37285519,PREF. CASTANHAL,14.3358,0.0,False,False,True
678645,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,F,871.006.303-04,GILSON FERREIRA ALVES,68551255,CCB,2024-12-05,2024-12-09,2031-10-20,NaN,45155052082,182399689,3.51,32.5,4.2463,0.0,0.0,NAO,NAO,2025-07-14,0.0000,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,0,NaN,NaN,NaN,45155052,PREF. TERESINA,4.2463,0.0,False,False,True


# Calcular % PDD em função de variáveis categóricas

In [68]:
cat_cols = ['Situacao', 'CedenteNome', 'SAC_TIPO_PESSOA', 'PagamentoParcial', 'TipoAtivo',
            '_MuitosContratos', '_MuitosEntes', 'Convênio']

pdd_ref = (1- df_final2['_ValorLiquido'].sum() / df_final2['ValorPresente'].sum()) * 100
print(f'{pdd_ref:_.2f}')

for col in cat_cols:
    aux_ = df_final2.groupby(col)[['_ValorLiquido', 'ValorPresente']].sum()/1e6
    aux_['%PDD'] = (1- aux_['_ValorLiquido'] / aux_['ValorPresente']) * 100
    
    if col == 'Convênio':
        aux_ = aux_.sort_values('ValorPresente', ascending=False)
    
    display(aux_[:20])
    

19.97


,_ValorLiquido,ValorPresente,%PDD
Situacao,,,
Aditado,7.851224,8.081707,2.851911
Sem cobranÃ§a,151.600857,191.150441,20.690292


,_ValorLiquido,ValorPresente,%PDD
CedenteNome,,,
BANCO DIGIMAIS S.A.,0.246985,0.297263,16.913659
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,159.205096,198.934885,19.971253


,_ValorLiquido,ValorPresente,%PDD
SAC_TIPO_PESSOA,,,
F,130.019122,150.625730,13.680669
J,29.432959,48.606418,39.446353


,_ValorLiquido,ValorPresente,%PDD
PagamentoParcial,,,
NAO,159.39059,198.993242,19.901506
SIM,0.06149,0.238906,74.261624


,_ValorLiquido,ValorPresente,%PDD
TipoAtivo,,,
CCB,150.961799,190.277251,20.662192
CT - Contrato,8.490281,8.954897,5.188396


,_ValorLiquido,ValorPresente,%PDD
_MuitosContratos,,,
False,159.179279,198.940162,19.986353
True,0.272802,0.291985,6.570138


,_ValorLiquido,ValorPresente,%PDD
_MuitosEntes,,,
False,159.436245,199.208493,19.965137
True,0.015835,0.023654,33.054812


,_ValorLiquido,ValorPresente,%PDD
Convênio,,,
GOV. MARANHAO,41.484712,46.568145,10.916117
GOV. ALAGOAS,32.999110,34.786090,5.137055
GOV. GOIAS,13.734775,17.185891,20.081099
PREF. BARREIRAS,9.304598,9.363838,0.632651
PREF. CABO FRIO,2.354433,6.814405,65.449174
GOV. RIO GRANDE DO NORTE,6.043314,6.765837,10.678985
PREF. CAMPOS DOS GOYTACAZES,4.330700,6.313879,31.409832
PREF. BALSAS,5.092831,5.119877,0.528237
GOV. TOCANTINS,2.902283,5.105860,43.157807


# Repetir o item anterior para vencidos

In [69]:
venc_ref = (df_final2['_ValorVencido'].sum() / df_final2['ValorPresente'].sum()) * 100
print(f'{venc_ref:_.2f}')

for col in cat_cols:
    aux_ = df_final2.groupby(col)[['_ValorVencido', 'ValorPresente']].sum()/1e6
    aux_['%Vencido'] = (aux_['_ValorVencido'] / aux_['ValorPresente']) * 100
    
    if col == 'Convênio':
        aux_ = aux_.sort_values('ValorPresente', ascending=False)
  
    display(aux_)
    

5.12


,_ValorVencido,ValorPresente,%Vencido
Situacao,,,
Aditado,0.038819,8.081707,0.480327
Sem cobranÃ§a,10.165953,191.150441,5.318299


,_ValorVencido,ValorPresente,%Vencido
CedenteNome,,,
BANCO DIGIMAIS S.A.,0.019911,0.297263,6.698064
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,10.184861,198.934885,5.119696


,_ValorVencido,ValorPresente,%Vencido
SAC_TIPO_PESSOA,,,
F,6.282185,150.625730,4.170725
J,3.922586,48.606418,8.070099


,_ValorVencido,ValorPresente,%Vencido
PagamentoParcial,,,
NAO,9.967889,198.993242,5.009160
SIM,0.236883,0.238906,99.153147


,_ValorVencido,ValorPresente,%Vencido
TipoAtivo,,,
CCB,9.932005,190.277251,5.219754
CT - Contrato,0.272766,8.954897,3.046001


,_ValorVencido,ValorPresente,%Vencido
_MuitosContratos,,,
False,10.197323,198.940162,5.125824
True,0.007448,0.291985,2.550866


,_ValorVencido,ValorPresente,%Vencido
_MuitosEntes,,,
False,10.202956,199.208493,5.121748
True,0.001815,0.023654,7.672974


,_ValorVencido,ValorPresente,%Vencido
Convênio,,,
GOV. MARANHAO,0.909802,46.568145,1.953700
GOV. ALAGOAS,1.368971,34.786090,3.935397
GOV. GOIAS,1.110039,17.185891,6.459012
PREF. BARREIRAS,0.018141,9.363838,0.193732
PREF. CABO FRIO,0.873612,6.814405,12.820073
GOV. RIO GRANDE DO NORTE,0.177908,6.765837,2.629507
PREF. CAMPOS DOS GOYTACAZES,0.449292,6.313879,7.115950
PREF. BALSAS,0.009430,5.119877,0.184182
GOV. TOCANTINS,0.499738,5.105860,9.787537


# Sacados em mais de um ente?

In [29]:
# SacadoCnpjCpf , SacadoNome
sacados_entes = df_final2.groupby('SacadoCnpjCpf')['Convênio'].agg([pd.Series.nunique, pd.unique])
sacados_entes.sort_values('nunique', ascending=False)[:35]

,nunique,unique
SacadoCnpjCpf,,
048.692.204-90,3,"[GOV. RIO GRANDE DO NORTE, GOV. GOIAS, PREF. T..."
323.026.574-20,3,"[GOV. RIO GRANDE DO NORTE, GOV. GOIAS, GOV. SÃ..."
292.798.363-15,2,"[GOV. MARANHAO, PREF. BALSAS]"
443.808.794-53,2,"[GOV. RIO GRANDE DO NORTE, PREF. SÃO JOSÉ DE M..."
813.733.763-68,2,"[GOV. MARANHAO, PREF. BALSAS]"
647.592.703-10,2,"[GOV. MARANHAO, PREF. SÃO LUIS]"
082.249.978-98,2,"[GOV. RIO GRANDE DO NORTE, PREF. SÃO JOSÉ DE M..."
054.906.964-00,2,"[GOV. RIO GRANDE DO NORTE, PREF. CAMPOS DOS GO..."
268.919.048-62,2,"[PREF. MAUÁ, PREF. SANTO ANDRE]"


In [101]:
df_final2.columns

Index(['Situacao', 'PES_TIPO_PESSOA', 'CedenteCnpjCpf',
       'TIT_CEDENTE_ENT_CODIGO', 'CedenteNome', 'Cnae', 'SecaoCNAEDescricao',
       'NotaPdd', 'SAC_TIPO_PESSOA', 'SacadoCnpjCpf', 'SacadoNome',
       'IdTituloVortx', 'TipoAtivo', 'DataEmissao', 'DataAquisicao',
       'DataVencimento', 'NumeroBoleto', 'NumeroTitulo', 'CampoChave',
       'ValorAquisicao', 'ValorNominal', 'ValorPresente', 'PDDNota',
       'PDDVencido', 'PagamentoParcial', 'Coobricacao', 'DataGeracao',
       'PDDTotal', 'CampoAdicional1', 'CampoAdicional2', 'CampoAdicional3',
       'CampoAdicional4', 'CampoAdicional5', 'PDDEfeitoVagao',
       'PercentagemEfeitoVagao', 'IdTituloVortxOriginador', 'Registradora',
       'IdContratoRegistradora', 'IdTituloRegistradora', 'CCB', 'Convênio',
       '_ValorLiquido'],
      dtype='object')

In [148]:
df_final2.sample(2)

,Situacao,PES_TIPO_PESSOA,CedenteCnpjCpf,TIT_CEDENTE_ENT_CODIGO,CedenteNome,Cnae,SecaoCNAEDescricao,NotaPdd,SAC_TIPO_PESSOA,SacadoCnpjCpf,SacadoNome,IdTituloVortx,TipoAtivo,DataEmissao,DataAquisicao,DataVencimento,NumeroBoleto,NumeroTitulo,CampoChave,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PagamentoParcial,Coobricacao,DataGeracao,PDDTotal,CampoAdicional1,CampoAdicional2,CampoAdicional3,CampoAdicional4,CampoAdicional5,PDDEfeitoVagao,PercentagemEfeitoVagao,IdTituloVortxOriginador,Registradora,IdContratoRegistradora,IdTituloRegistradora,CCB,Convênio,_ValorLiquido,_ValorVencido,_MuitosContratos
608870,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,J,565.131.602-78,ISABEL GIANE COELHO RODRIGUES,100015359,CCB,2025-06-13,2025-06-16,2030-07-15,NaN,57139775060,57139775060,125.10,931.37,128.9142,0.0,0.0,NAO,NAO,2025-07-14,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,57139775,PREF. JURUTI,128.9142,0.0,False
343873,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,J,490.381.254-53,JOACILDO DE MEDEIROS GALVAO,41173845,CCB,2024-05-29,2024-05-31,2028-11-16,NaN,3620565153,155164184,8.88,30.39,12.0912,0.0,0.0,NAO,NAO,2025-07-14,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,36205651,GOV. RIO GRANDE DO NORTE,12.0912,0.0,False


# Verificação de datas

In [54]:
(df_final2['DataEmissao'] > df_final2['DataAquisicao']).sum() # ok

0

In [55]:
(df_final2['DataAquisicao'] > df_final2['DataVencimento']).sum() # ok

0

# Verificação dos valores

In [57]:
(df_final2['ValorAquisicao'] > df_final2['ValorNominal']).sum() # ok

3696

In [58]:
(df_final2['ValorAquisicao'] > df_final2['ValorPresente']).sum() # ok

3696

In [59]:
(df_final2['ValorPresente'] > df_final2['ValorNominal']).sum() # ok

0

In [64]:
mask = df_final2['ValorAquisicao'] > df_final2['ValorNominal']
display(df_final2[mask]['PagamentoParcial'].value_counts())
df_final2[mask].sample(3)

PagamentoParcial
SIM    3696
Name: count, dtype: int64

,Situacao,PES_TIPO_PESSOA,CedenteCnpjCpf,TIT_CEDENTE_ENT_CODIGO,CedenteNome,Cnae,SecaoCNAEDescricao,NotaPdd,SAC_TIPO_PESSOA,SacadoCnpjCpf,SacadoNome,IdTituloVortx,TipoAtivo,DataEmissao,DataAquisicao,DataVencimento,NumeroBoleto,NumeroTitulo,CampoChave,ValorAquisicao,ValorNominal,ValorPresente,PDDNota,PDDVencido,PagamentoParcial,Coobricacao,DataGeracao,PDDTotal,CampoAdicional1,CampoAdicional2,CampoAdicional3,CampoAdicional4,CampoAdicional5,PDDEfeitoVagao,PercentagemEfeitoVagao,IdTituloVortxOriginador,Registradora,IdContratoRegistradora,IdTituloRegistradora,CCB,Convênio,_ValorLiquido,_ValorVencido,_MuitosContratos,_MuitosEntes,CPF_válido
217716,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,J,452.961.623-15,PAULO HENRIQUE PINTO BERREDO,35169955,CCB,2024-02-21,2024-02-27,2024-12-02,NaN,3263847509,31593418,1176.96,346.44,346.44,0.0,346.44,SIM,NAO,2025-07-14,346.44,NaN,NaN,NaN,NaN,NaN,346.44,1.0,35169181,NaN,NaN,NaN,32638475,GOV. MARANHAO,0.00,346.44,False,False,True
38683,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,F,883.234.983-34,ARIALDO LUSTOSA RIOS,61804657,CCB,2024-10-25,2024-10-25,2025-06-25,NaN,42854321007,181679386,133.98,3.01,3.01,0.0,0.00,SIM,NAO,2025-07-14,0.00,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0,NaN,NaN,NaN,42854321,GOV. MARANHAO,3.01,3.01,False,False,True
297174,Sem cobranÃ§a,J,34.337.707/0001-00,318853,BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,6499999,OUTRAS ATIVIDADES DE SERVIÃOS FINANCEIROS NÃ...,AA,F,041.610.183-63,ADRIANO AURELIO DE MENEZES BRAGA,61077836,CCB,2024-10-22,2024-10-23,2025-06-25,NaN,42703028007,181653897,273.36,52.18,52.18,0.0,0.00,SIM,NAO,2025-07-14,0.00,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0,NaN,NaN,NaN,42703028,GOV. MARANHAO,52.18,52.18,False,False,True


In [70]:
df_final2.columns

Index(['Situacao', 'PES_TIPO_PESSOA', 'CedenteCnpjCpf',
       'TIT_CEDENTE_ENT_CODIGO', 'CedenteNome', 'Cnae', 'SecaoCNAEDescricao',
       'NotaPdd', 'SAC_TIPO_PESSOA', 'SacadoCnpjCpf', 'SacadoNome',
       'IdTituloVortx', 'TipoAtivo', 'DataEmissao', 'DataAquisicao',
       'DataVencimento', 'NumeroBoleto', 'NumeroTitulo', 'CampoChave',
       'ValorAquisicao', 'ValorNominal', 'ValorPresente', 'PDDNota',
       'PDDVencido', 'PagamentoParcial', 'Coobricacao', 'DataGeracao',
       'PDDTotal', 'CampoAdicional1', 'CampoAdicional2', 'CampoAdicional3',
       'CampoAdicional4', 'CampoAdicional5', 'PDDEfeitoVagao',
       'PercentagemEfeitoVagao', 'IdTituloVortxOriginador', 'Registradora',
       'IdContratoRegistradora', 'IdTituloRegistradora', 'CCB', 'Convênio',
       '_ValorLiquido', '_ValorVencido', '_MuitosContratos', '_MuitosEntes',
       'CPF_válido'],
      dtype='object')